## Note: MySQL OpenGIS for Nearest Neighbour Query / Proximity Search

Objective is to leverage PostGIS for NNQ.

This notebook demonstrates NNQ by using MsSQL OpenGIS implementation.

Query: find the list of restaurants that serve $cuisine and is within Xkm from a location (Lat, Lon)

Setup

In [12]:
%pip install --user PyMySQL
%pip install --user pymysql[rsa]
%pip install --user cryptography

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pymysql
import pandas as pd

In [2]:
def getConfigOrDefault(config_filename, config,label,default=None):
    if config.get(label) is not None:
        return config.get(label)
    if default is not None:
        return default
    print(f"{config_filename} file does not have {label} parameter!")
    return None

In [3]:
from dotenv import dotenv_values

config_filename = './mysql.cfg'
config = dotenv_values(config_filename)

USER_NAME = getConfigOrDefault(config_filename, config, 'MYSQL_UNAME')
USER_PWD  = getConfigOrDefault(config_filename, config, 'MYSQL_UPWD')
MYSQL_HOST = getConfigOrDefault(config_filename, config, 'MYSQL_HOST', 'localhost')
MYSQL_PORT = getConfigOrDefault(config_filename, config, 'MYSQL_PORT', '3306')
MYSQL_DB = getConfigOrDefault(config_filename, config, 'MYSQL_DBNAME', 'test_spatial')

if USER_PWD is None or USER_PWD is None:
    print(f"{config_filename} file does not have parameters: MYSQL_UNAME and/or MYSQL_UPWD!")

In [4]:
import pymysql

# Database connection details
db_host = MYSQL_HOST      # IP Address
db_user = USER_NAME       # MySQL username
db_password = USER_PWD    # MySQL password
db_name = MYSQL_DB        # Database name

# Establish a persistent connection
conn = pymysql.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name,
    cursorclass=pymysql.cursors.DictCursor  # Fetch results as dictionaries
)

print("Connected to MySQL database")

Connected to MySQL database


CREATE TABLE locations (
  id INT AUTO_INCREMENT PRIMARY KEY,
  name VARCHAR(255),
  coordinates POINT
);

In [28]:
query="""
CREATE TABLE restaurants (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    coordinates POINT
);
"""
response = None
try:
    with conn.cursor() as cursor:
        response = cursor.execute(query)
        conn.commit()  # Commit changes for INSERT/UPDATE/DELETE

except pymysql.MySQLError as e:
    print(f"Query Error: {e}")

print(response)


0


In [5]:
query="""
show tables;
"""
response = None
try:
    with conn.cursor() as cursor:
        cursor.execute(query)
        response = cursor.fetchall()
        conn.commit()  # Commit changes for INSERT/UPDATE/DELETE

except pymysql.MySQLError as e:
    print(f"Query Error: {e}")

print(response)

[{'Tables_in_test_spatial': 'restaurants'}]


In [ ]:
query="""
DROP TABLE restaurants ;
"""
response = None
try:
    with conn.cursor() as cursor:
        response = cursor.execute(query)
        conn.commit()  # Commit changes

except pymysql.MySQLError as e:
    print(f"Query Error: {e}")

print(response)

In [16]:
response = conn.close()
print(response)

None


In [31]:
dml_insert="""
INSERT INTO restaurants (name, coordinates)
VALUES ('First Duck', ST_PointFromText('POINT(72.8777 19.0760)'));
"""
response = None
try:
    with conn.cursor() as cursor:
        response = cursor.execute(dml_insert)
        conn.commit()  # Commit changes for INSERT/UPDATE/DELETE

except pymysql.MySQLError as e:
    print(f"Query Error: {e}")

print(response)

1


In [35]:
def execute_sqL_with_fetchX(sql_stmt,x=0):
    """ Executes SQL statement with fetchnone
        Not worried about performance
    """
    print(f"Executing SQL statement with fetchX-{sql_stmt,x}")
    response = None
    try:
        # Connect to an existing database
        connection = pymysql.connect(
                host=db_host,
                user=db_user,
                password=db_password,
                database=db_name,
                cursorclass=pymysql.cursors.DictCursor  # Fetch results as dictionaries
        )

        # Create a cursor to perform database operations
        cursor = connection.cursor()
        # Executing given SQL query
        response = cursor.execute(sql_stmt)
        connection.commit()

        if x == 1:
            response = cursor.fetchone()
        elif x == 2:
            response = cursor.fetchall()
        else:
            pass

        cursor.close()
        connection.close()

    except (Exception, Error) as error:
        response = f"Error : {error}"
        print(response)
    finally:
        if (connection):
            #cursor.close()
            #connection.close()
            print(f"Connection to MySQL({MYSQL_HOST}:{MYSQL_PORT}) is closed")
    return response

In [36]:
print(execute_sqL_with_fetchX("SELECT * FROM restaurants",2))

Executing SQL statement with fetchX-('SELECT * FROM restaurants', 2)
Connection to MySQL(127.0.0.1:5432) is closed
[{'id': 1, 'name': 'First Duck', 'coordinates': b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xc0\xec\x9e<,8R@\xfa~j\xbct\x133@'}, {'id': 2, 'name': 'First Duck', 'coordinates': b'\x00\x00\x00\x00\x01\x01\x00\x00\x00\xc0\xec\x9e<,8R@\xfa~j\xbct\x133@'}]


In [ ]:
print(execute_sqL_with_fetchX("DROP TABLE IF EXISTS restaurants",0))

Executing SQL statement with fetchX-('DROP TABLE IF EXISTS restaurants', 0)


In [40]:
print(execute_sqL_with_fetchX("SHOW TABLES;",2))

Executing SQL statement with fetchX-('SHOW TABLES;', 2)
Connection to MySQL(127.0.0.1:5432) is closed
[{'Tables_in_test_spatial': 'restaurants'}]


In [39]:
query="""
CREATE TABLE restaurants (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    cuisine VARCHAR(63),
    longitude FLOAT,
    latitude FLOAT,
    location POINT
);
"""
print(execute_sqL_with_fetchX(query,0))

Executing SQL statement with fetchX-('\nCREATE TABLE restaurants (\n    id INT AUTO_INCREMENT PRIMARY KEY,\n    name VARCHAR(255),\n    cuisine VARCHAR(63),\n    longitude FLOAT,\n    latitude FLOAT,\n    location POINT\n);\n', 0)
Connection to MySQL(127.0.0.1:5432) is closed
0


In [41]:
filename="./restaurants.csv"
df_loaded = pd.read_csv(filename)
df_loaded[:10]

,Name,Lon,Lat
0,Morris Park Bake Shop,-73.856077,40.848447
1,Wendy'S,-73.961704,40.662942
2,Riviera Caterer,-73.982420,40.579505
3,Tov Kosher Kitchen,-73.860115,40.731174
4,Brunos On The Boulevard,-73.880383,40.764312
5,Dj Reynolds Pub And Restaurant,-73.985136,40.767692
6,Wilken'S Fine Food,-73.906851,40.619903
7,Regina Caterers,-74.005289,40.628886
8,Taste The Tropics Ice Cream,-73.948261,40.640827
9,Kosher Island,-74.137729,40.611957


In [42]:
import random
def assignCuisineRandomly(df,cuisine,cname):
    for i in range(df.shape[0]):
        df.at[i,cname] = cuisine[random.randint(0,len(cuisine)-1)]

In [43]:
cuisine = ['italian', 'chinese', 'french', 'zambian', 'egyptian', 'canadian', 'mexican', 'vietnamese', 'cajun', 'korean', 'thai', 'brazilian','colombian','peruvian','ecuadorian', 'japanese','indian','malaysian','russian', 'indonesian']
assignCuisineRandomly(df_loaded,cuisine,'Cuisine')
df_loaded[-4:]

,Name,Lon,Lat,Cuisine
4996,Wagner College - Hawk' Nest,-74.092853,40.615121,thai
4997,Ellen Deli & Grocery,-74.007810,40.725708,peruvian
4998,Crepes On Columbus,-73.961831,40.801052,brazilian
4999,Capital Grille,-73.974723,40.751244,italian


In [61]:
def formatSQL(df):
    name = df['Name'].replace("'"," ")
    sql_stmt = f"INSERT INTO test_spatial.restaurants (name, cuisine, longitude, latitude, location) VALUES ('{name}','{df['Cuisine']}',{df['Lon']},{df['Lat']},ST_PointFromText('POINT({df['Lon']} {df['Lat']})', 4326));"

    return sql_stmt

In [62]:
#print(formatSQL(df_loaded[0]))
idx=4996
print(type(df_loaded.loc[idx].to_dict()))
df_loaded.loc[idx].to_dict()
print(formatSQL(df_loaded.loc[idx].to_dict()))

<class 'dict'>
INSERT INTO test_spatial.restaurants (name, cuisine, longitude, latitude, location) VALUES ('Wagner College - Hawk  Nest','thai',-74.09285299999999,40.61512099999999,ST_PointFromText('POINT(-74.09285299999999 40.61512099999999)', 4326));


In [51]:
# Load data to test_spatial.restaurants table
# using ST_PointFromText('POINT($longitude $latitude)')
try:
    # Connect to an existing database
    connection = pymysql.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=db_name,
        cursorclass=pymysql.cursors.DictCursor  # Fetch results as dictionaries
    )

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")
    sql_stmt = ""
    for i in range(df_loaded.shape[0]):
        #addARestaurant(connection, df_loaded.loc[i].to_dict())
        sql_stmt = formatSQL(df_loaded.loc[i].to_dict())
        #print(sql_stmt)
        cursor.execute(sql_stmt)

    connection.commit()

    #Closing the connection
    connection.close()

except (Exception, Error) as error:
    print("Error: ", error)
finally:
    #if (connection):
    #cursor.close()
    #connection.close()
    print("MySQL connection is closed")

You are connected to -  {'version()': '8.0.44'} 

MySQL connection is closed


## Query

In [59]:
NNQ_LON = -74.092853
NNQ_LAT =  40.615121
NNQ_RADIUS = 2000
query = f"""
SELECT name
    , cuisine
    , longitude
    , latitude
    , ST_Distance(location, ST_PointFromText('POINT({NNQ_LON} {NNQ_LAT})',4326)) AS dist2NNQ
FROM test_spatial.restaurants
WHERE ST_Distance(location, ST_PointFromText('POINT({NNQ_LON} {NNQ_LAT})',4326),'metre') < {NNQ_RADIUS};
"""

explain_query = "Explain "+ query

In [60]:
reply = execute_sqL_with_fetchX(query,2)
print(type(reply))

Executing SQL statement with fetchX-("\nSELECT name\n    , cuisine\n    , longitude\n    , latitude\n    , ST_Distance(location, ST_PointFromText('POINT(-74.092853 40.615121)',4326)) AS dist2NNQ\nFROM test_spatial.restaurants\nWHERE ST_Distance(location, ST_PointFromText('POINT(-74.092853 40.615121)',4326),'metre') < 2000;\n", 2)
Connection to MySQL(127.0.0.1:5432) is closed


NameError: name 'Error' is not defined

## Spatial Index